#### Import Libraries

In [121]:
import os

import pandas as pd
import numpy as np

from openai import OpenAI

from sklearn.cluster import KMeans
from dotenv import load_dotenv
load_dotenv()

#### Read in data

In [27]:
notes_path = r'C:\Users\Joslyn\MSAI\AI in Healthcare\mimic-iii-clinical-database-1.4\NOTEEVENTS.csv.gz'
notes = pd.read_csv(notes_path)

C:\Users\Joslyn\AppData\Local\Temp\ipykernel_2556\1557264638.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv(notes_path)


#### Explore data

In [28]:
notes.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT'],
      dtype='object')

In [29]:
notes.CATEGORY.value_counts()

CATEGORY
Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: count, dtype: int64

In [30]:
notes.DESCRIPTION.value_counts()

DESCRIPTION
Report                                                 1132519
Nursing Progress Note                                   191836
CHEST (PORTABLE AP)                                     169270
Physician Resident Progress Note                         62698
CHEST (PA & LAT)                                         43158
                                                        ...   
RP FOOT 1 VIEW RIGHT PORT                                    1
Intensvist                                                   1
O IVP NO TOMO IN O.R.                                        1
OPL KNEE (2 VIEWS) IN O.R. PORT LEFT                         1
RO HIP NAILING IN OR W/FILMS & FLUORO RIGHT IN O.R.          1
Name: count, Length: 3848, dtype: int64

In [31]:
sample_note = notes.loc[1, "TEXT"]
print(sample_note)

Admission Date:  [**2118-6-2**]       Discharge Date:  [**2118-6-14**]

Date of Birth:                    Sex:  F

Service:  MICU and then to [**Doctor Last Name **] Medicine

HISTORY OF PRESENT ILLNESS:  This is an 81-year-old female
with a history of emphysema (not on home O2), who presents
with three days of shortness of breath thought by her primary
care doctor to be a COPD flare.  Two days prior to admission,
she was started on a prednisone taper and one day prior to
admission she required oxygen at home in order to maintain
oxygen saturation greater than 90%.  She has also been on
levofloxacin and nebulizers, and was not getting better, and
presented to the [**Hospital1 18**] Emergency Room.

In the [**Hospital3 **] Emergency Room, her oxygen saturation was
100% on CPAP.  She was not able to be weaned off of this
despite nebulizer treatment and Solu-Medrol 125 mg IV x2.

Review of systems is negative for the following:  Fevers,
chills, nausea, vomiting, night sweats, change in we

In [49]:
ecg_notes = notes[notes['CATEGORY'] == 'ECG']
ecg_notes.head()
unhealthy_ecg = ecg_notes.iloc[0,-1]
healthy_ecg = ecg_notes.iloc[3,-1]
print(unhealthy_ecg, healthy_ecg)

Sinus tachycardia
Short PR interval
Possible anterior infarct - age undetermined
Left atrial abnormality
Inferior T wave changes are borderline
Repolarization changes may be partly due to rate
Low QRS voltages in limb leads
Since previous tracing of [**2103-7-27**], no significant change

 Sinus rhythm. Normal ECG. Compared to the previous tracing of [**2145-12-12**] the rate
has slowed. QRS amplitude has increased. Otherwise, no change.




#### LLM prompting - chat completion

In [33]:
api_key = os.environ.get("OPENAI_API_KEY")

Let's start with a simple prompt

In [168]:
zero_shot_prompt = """
Read the information of each separate echocardiogram note and 
decide if each individual person is 'healthy' or 'unhealthy'. 
ECG1: {} . ECG2: {}.

health-status1:
health-status2:
""".format(unhealthy_ecg, healthy_ecg)

In [150]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": zero_shot_prompt}]
)
content = response.choices[0].message.content
print(content)

health-status1: Unhealthy
health-status2: Healthy


It got it right, but let's give it more context to improve

In [53]:
sys_message = """
You are a helpful medical expert with vast knowledge on reading echocardiograms 
and evaluating the severity of arrythmias.
"""

In [151]:
client = OpenAI()
response_w_context = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": sys_message},
        {"role": "user", "content": zero_shot_prompt}
    ]
)
content = response.choices[0].message.content
print(content)

health-status1: Unhealthy
health-status2: Healthy


In [166]:
few_shot_prompt = """
Q: Normal sinus rhythm. P-wave lasts 0.1 seconds. PR interval is 0.15 seconds. Heart rate 80 beats per minute
A: Healthy
Q: Ventricular tachycardia. T-wave abnormalize. Slight av conduction delay.
A: Unhealthy
Q: {}
A:
""".format(unhealthy_ecg)

In [167]:
response3 = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": sys_message},
        {"role": "user", "content": few_shot_prompt}
    ]
)
content = response3.choices[0].message.content
print(content)

Unhealthy


Now let's try chain of thought prompting

In [164]:
cot_prompt = """
Q: Normal sinus rhythm. P-wave lasts 0.1 seconds. PR interval is 0.15 seconds. Heart rate 80 beats per minute. Slight av conduction delay.
A: Normal sinus rhythm is a good sign. PR interval is within desired range. Heart rate is a little elevated but alright. Conduction delay worth monitoring but not very converning
Classification: Healthy
Q: Atrial fibrillation. T-wave abnormality. Short PR interval.
A: Atrial fibrillation is very concerning, this needs to be addressed. T-wave abnormalizy should be monitored. Short PR interval is not ideal but not too concerning.
Classification: Unhealthy
Q: {}
A:
""".format(unhealthy_ecg)

In [165]:
response4 = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": sys_message},
        {"role": "user", "content": cot_prompt}
    ]
)
content = response4.choices[0].message.content
print(content)

There are several abnormalities detected in this case. Sinus tachycardia indicates higher than normal heart rate. Short PR interval may suggest pre-excitation syndrome like Wolff-Parkinson-White. An undetermined anterior infarct indicates possible previous heart attack. Left atrial abnormality can suggest a number of conditions including atrial enlargement due to high blood pressure or valve disease. Borderline T wave changes in the inferior leads might indicate ischemia. Repolarization changes due to rate suggest possible non-specific T wave changes. Low QRS voltages in the limb leads may indicate a thinning of the heart muscle. No change since the previous tracing suggest these are not acute changes. 
Classification: Unhealthy



Lastly lets do tree of thought prompting

In [180]:
tot_prompt = """
Imagine you have three different doctors in a room evaluating an ECG note. 
All experts write down 1 step of their thinking then share it with the group.
If disagreement occurs, they deliberate and a majority vote decides. 
Then all experts go on to the next step, countinuing until they reach the answer, 
sharing the logical reasoning along the way. 
In the event that an expert realizes they were explicitly incorrect, they leave the room.
The goal is for the doctors to classify the findings of the ecg to be healthy or unhealthy.
This final classification will be clearly stated in the last line of the conversation.
The ecg note is:
{}
""".format(unhealthy_ecg)

In [181]:
response5 = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": sys_message},
        {"role": "user", "content": tot_prompt}
    ]
)
content = response5.choices[0].message.content
print(content)

Doctor 1: The first observation from the ECG is that it shows us sinus tachycardia. This is a regular heart rhythm, but it's fast — over 100 beats per minute. It might be a result of numerous conditions such as physical exertion, mental stress, fever, or anemia, and so it's not necessarily suggestive of being unhealthy on its own.

Doctor 2: I agree with your observation. However, the ECG also reveals a short PR interval. This could suggest a condition like Wolff-Parkinson-White syndrome which is a serious heart condition. Though not all patients with short PR intervals have serious heart conditions, it should be investigated further in combination with the patient's history and other findings.

Doctor 3: There's also a notation of a possible anterior infarct, but the age is undetermined. The impact of this depends on when it occurred, as a recent infarct presents a very different risk profile than an old one. We also need to consider the repolarization changes, which could be related 

#### LLM embeddings

Our goal is to get the embeddings of each note in a sample of notes and cluster them to predict healthy or unhealthy based on semantics of each note.

In [88]:
def get_embedding(text, model="text-embedding-3-small"):
    """
    Get the embedding vector for a given text using a specified model.

    Args:
        text (str): The input text for which the embedding vector is to be generated.
        model (str, optional): The name of the model to use for generating embeddings.
            Defaults to "text-embedding-3-small".

    Returns:
        numpy.ndarray: The embedding vector for the input text.
    """
    embeds = client.embeddings.create(input=[text], model=model)
    return embeds.data[0].embedding

In [131]:
sample_notes = ecg_notes.iloc[:10]

In [132]:
sample_notes.loc[:,'embedding'] = sample_notes['TEXT'].apply(get_embedding)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.009251023642718792, 0.01653558760881424, 0.02027062699198723, 0.019949065521359444, -0.02607106789946556, 0.028742486611008644, 0.0005372210871428251, -0.04031863436102867, -0.012924225069582462, -0.049520187079906464, -0.014643331989645958, 0.009776649996638298, -0.005577823147177696, 0.034975796937942505, 0.03411005809903145, -0.05837544426321983, 0.006697098258882761, -0.003917462192475796, 0.023350177332758904, 0.02054271474480629, -0.00897275097668171, -0.025131123140454292, 0.0594143308699131, -0.0199366994202137, 0.03415952995419502, -0.007834924384951591, -0.008521330542862415, -0.030869727954268456, 0.019305946305394173, -0.00579425785690546, 0.030251342803239822, -0.029509281739592552, 0.006047795061022043, -0.003552615875378251, -0.0420253723859787, 0.032057024538517, -0.05144954472780228, -0.006783672142773867, -0.029088782146573067, -0.04286637529730797, 0.04741768166422844, -0.027629395946860313, -0.0557040274143219, -

C:\Users\Joslyn\AppData\Local\Temp\ipykernel_2556\2642219950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_notes.loc[:,'embedding'] = sample_notes['TEXT'].apply(get_embedding)


#### Perform clustering on embeddings

In [133]:
matrix = np.vstack(sample_notes['embedding'].values)
clf = KMeans(random_state=10, n_clusters=2, init='k-means++')
clf.fit(matrix)
sample_notes.loc[:, 'cluster'] = clf.labels_


C:\Users\Joslyn\AppData\Local\Temp\ipykernel_2556\1565938831.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_notes.loc[:, 'cluster'] = clf.labels_


In [134]:
sample_notes['cluster'].value_counts()

cluster
0    7
1    3
Name: count, dtype: int64

#### Unsupervised Learning performance exploration

Since we don't have labeled data to calculate performance, lets look at the split and see if we can identify it as reasonable

In [141]:
cluster0 = sample_notes[sample_notes["cluster"] == 0]
cluster1 = sample_notes[sample_notes["cluster"] == 1]

In [142]:
cluster0.TEXT, cluster1.TEXT

(100164    Sinus rhythm with A-V conduction delay.  Infer...
 100165    Sinus rhythm with A-V conduction delay. Infero...
 100166    Sinus rhythm. Normal ECG. Compared to the prev...
 100169    Sinus rhythm.  Left axis deviation.  Probable ...
 100170    Sinus rhythm\nLeft axis deviation - anterior f...
 100171    Sinus bradycardia. Left anterior fascicular bl...
 100172    Sinus rhythm\nLeft anterior fascicular block\n...
 Name: TEXT, dtype: object,
 52120     Sinus tachycardia\nShort PR interval\nPossible...
 100167    Sinus tachycardia. Low limb lead voltage. Comp...
 100168    Sinus bradycardia\nPremature beats - may be ju...
 Name: TEXT, dtype: object)

I am no ECG expert, but from a linguistic perspective this seems like a resonable split. 
It appear that cluster 0 is heathy and cluster 1 is unhealthy.
The sinus bradycardia could potentially still be classified as healthy but overall it looks to have performed relatively well.